In [ ]:
%gui wx

from PYME.LMVis import VisGUI

In [ ]:
# Delay so wx can boot up
import time
time.sleep(8)

In [ ]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline
recipe = pipeline.recipe

In [ ]:
from ch_shrinkwrap import shape
from PYME.IO import tabular
import numpy as np

# Create a torus
R = 50
r = 30
t = shape.Torus(R, r)
pts_t = t.points(1)
# pts_t = np.load('hemo_pts.npy')

# Create a bounding box
# import numpy as np
# pts_t = np.vstack([pts_t, np.array([-2*R,-2*R,-2*R]), np.array([2*R,2*R,2*R])])

pipeline.addDataSource('points', tabular.mappingFilter({'x' : pts_t[:,0], 'y':pts_t[:,1], 'z':pts_t[:,2]}))
pipeline.selectDataSource('points')
pymevis.add_pointcloud_layer(ds_name='points')

In [ ]:
# Surface generation parameters
threshold_density = 0.005
# threshold_density = 0.0001
n_points_min = 50
# n_points_min = 100
repair = False
remesh = False

# Run surface generation recipe on the point cloud
from PYME.LMVis.layers.mesh import TriangleRenderLayer
from ch_shrinkwrap import _membrane_mesh as membrane_mesh
from ch_shrinkwrap import delaunay_utils
from scipy.spatial import Delaunay

d = Delaunay(pts_t)
tri = delaunay_utils.orient_simps(d, pts_t)
f = delaunay_utils.surf_from_delaunay(tri)

mesh = membrane_mesh.MembraneMesh(1.1*pts_t,f)

surf_name, surf_count = pipeline.new_ds_name('surf', return_count=True)
pipeline.dataSources[surf_name] = mesh
layer = TriangleRenderLayer(pipeline, dsname=surf_name, method='shaded', cmap = ['C', 'M', 'Y', 'R', 'G', 'B'][surf_count % 6])
pymevis.add_layer(layer)

In [ ]:
mesh.remesh()

In [ ]:
mesh.delaunay_remesh(pts_t)

In [ ]:
# layer.display_normals=True

In [ ]:
# from ch_shrinkwrap import sdf

In [ ]:
# v0 = np.array([0,0,0])
# v1 = np.array([1,0,0])
# v2 = np.array([1,1,0])
# v3 = np.array([0,0,1])
# v = np.vstack([v0,v1,v2,v3])

# sdf.tetrahedron(0.1*v,v0,v1,v2,v3)